**A Column Generation Approach for the Airline Crew Pairing Problem**

This laboratory session is about a subproblem of Airline Crew Scheduling, known in the literature
as the Airline Crew Pairing problem. Let us consider a flight leg which is a single takeofflanding phase to be carried out within a predefined time window, for example a flight that has
to take off from Rome airport FCO at 9:00 AM and has to land at Milan airport MPX at 10:15 AM. A pairing is a feasible sequence of flight legs that can be performed by the same crew members, starting and ending at the same airport. Organizing the daily flights into pairings is an important problem for airline companies, and it is generally followed by an assignment phase
of the crew members to each pairing (a problem known as Airline Crew Rostering). The pairings must be created so as to be feasible with respect to different constraints like, for instance, the
compatibility of the flight legs, minimum rest periods and maximum total flight time.

Given a set $I$ of flight legs, a feasible pairing can be represented by an incidence column
vector $A^j \in \mathbb{R}^I$, with component $a_{ij}$ equal to 1 if the flight leg $i \in I$ is included in the paring and 0 otherwise. Given


*   a set $I$ of flight legs,
*   the set $J$ of all feasible pairings described by the column vectors $A^j$ with $j \in J$,

the Airline Crew Pairing problem consists in selecting the minimum number of feasible pairings
that exactly cover all the flight legs. Note that minimizing the number of pairing amounts to
minimizing the number of crews needed to cover all the flight legs.

Given a set of binary variables $x_j$ with $j \in J$ indicating which feasible pairings are selected, the addressed problem can be formulated as the following set partitioning problem

\begin{array}{lll}
  \min & \sum_\limits{j \in J} x_j & & & \text{(1)}\\
  \textrm{s.t.} & Ax = 1 & & &(2)\\
  & x_{j} \in \{0,1\} &  j \in J, &  &(3)\\
\end{array}

where $A \in \mathbb{R}^{I \times J}$
is the incidence matrix of flight legs and pairings, containing all the feasible
pairing columns $A^j$. Since the number $|J|$ of columns grows exponentially with the number
$|I|$ of flight legs, it is impossible for real-world instances to construct the whole matrix $A$. For this reason a classical way to face this problem is the column generation method, in which the columns of $A$ are generated on the fly  only if needed, by exploiting the dual formulation of the
linear relaxation of problem (1)-(3).

In this laboratory session, we consider an instance of the Airline Crew Pairing problem which involves 9 airports and 42 flight legs. Each flight leg $i \in I$ is characterized by an origin airport $o_i$ and a destination airport $d_i$, a starting time $s_i$ and the length $l_i$ of the flight leg expressed
in hours. The feasible pairings must be created so that the starting and the ending airports coincide and so that there is at least 1 hour of rest period between any two consecutive legs.



1.   Propose a column generation strategy to solve the linear relaxation of the Airline Crew
Pairing problem.

**Hint**:  Exploit the structure of the dual formulation of the problem.



2.   Implement in Python a column generation algorithm and solve the linear relaxation of the master problem for the given instance, starting from a given feasible solution.

In [1]:
# When using Colab, make sure you run this instruction beforehand
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 3.9 MB/s eta 0:00:00


In [2]:
import mip
import pandas as pd # to handle the data of the problem
from mip import BINARY
import numpy as np
import math

In [3]:
data = pd.read_csv('data.csv',sep=';')

# MASTER PROBLEM
# DEFINITION OF SETS AND PARAMETERS

#number of flight legs
n_i = 42

n_q = 9

# set of the flight legs
I = range(n_i)
# set of the airports
Q = range(n_q)

# origin airport of the flight
o = data['o'].values

# destination airport of the flight
d = data['d'].values

# starting time of the flight
s = data['s'].values

# length of the flight
l = data['l'].values

#----------------------------------------
# initialize a starting feasible solution
#----------------------------------------
# number of pairings
n_j = int(n_i/2)

# pairings matrix
a = np.zeros((n_i,n_j))

# Valid pairings for the specific instance in data.csv
for i in I:
  a[i,math.floor(i/2)] = 1

# set of all possible edge 
K = [(i,i1) for i in I for i1 in I]

In [4]:
#----------------------------------------
# create the graph of feasible paths
#----------------------------------------
rest = 1
# TO-DO: modify the arc set A by adding an arc (i, i1) if destination of i coincides with the origin 
# and the time window constraints are satisfied
A =


In [5]:
# function to check if the origin/destination has at least 1 outgoing/ingoing arc
def check(A,index,pos):
  if pos >=0 and pos <=1 and isinstance(pos,int):
    found = False
    for t in A:
      if t[pos] == index:
        found = True
    return found
  else:
    return None

In [ ]:
found = True


while(found): # cycle until new feasible columns is found

  # set of pairings
  J = range(n_j)
  # SOLVE MASTER
  # Master problem definition
  master = mip.Model()
  # TO-DO
  # Master problem variables
  

  # Master problem objective
  
  
  # Master problem constraints
  

  
  # optimizing
  master.optimize()
  print('Solved Master pairings:')
  print(master.objective.x)
  # coefficients of the obj. of the pricing set to the shadow price of the master
  y_star = [master.constrs[i].pi for i in I]

  found = False

  # try a pair of flight legs source-target such that:
  # 1. the origin of the source is equal to the target destination
  # 2. the origin has at least 1 outgoing arc
  # 3. the destination has at least 1 ingoing arc

  for (src,trg) in K:
  
    if o[src] == d[trg] and check(A,src,0) and check(A,trg,1):

      found = False

      # SOLVE PRICING
      # Pricing problem definition
      pricing = mip.Model()
      # TO-DO
      # Pricing problem variables
            

      # Pricing problem objective
      

      # Pricing problem constraints
           
      for i in I:
        if i!= src and i!=trg:
          pricing.add_constr(mip.xsum(z[edge] for edge in A if edge[1]==i)-mip.xsum(z[edge] for edge in A if edge[0]==i) == 0)
      # activation
      for i in I:
        if i!= src and i!=trg:
          pricing.add_constr(mip.xsum(z[edge] for edge in A if edge[1]==i)== alpha[i])
      # optimizing
      pricing.optimize()

      reducedCost = 1 - pricing.objective.x

      if reducedCost < 0:
        # TO-DO: add the column to the master problem
        found = True
        break

In [ ]:
indices = []

for j in J:
  if x[j].x == 1:
    indices.append(j)
print("Selected %i pairings" % len(indices))
print(indices)
print(a[:,indices])